In [ ]:
import tensorflow as tf
import numpy as np
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Replace 'path/to/zip/file.zip' with the path to your zip file
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/archive_2.zip', 'r')

# Replace 'path/to/extract' with the directory where you want to extract the zip file
zip_ref.extractall('/content')

zip_ref.close()

In [ ]:
# Set the path to the dataset
data_dir = '/content/images'

In [ ]:
# Set the batch size and input image size for the model
batch_size = 32
input_shape = (224, 224, 3)


In [ ]:
# Set the number of classes in the dataset
num_classes = 4

In [ ]:
# Set the number of epochs to train the model for
num_epochs = 50

In [ ]:
# Set up data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Set up data augmentation for validation data
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Create data generators for training and validation data
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, ''),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, ''),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 15921 images belonging to 4 classes.
Found 15921 images belonging to 4 classes.


In [ ]:
# Load the MobileNetV2 model without the top layers
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
# Add a new top layer for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator,
          epochs=num_epochs,
          validation_data=val_generator,
          callbacks=[EarlyStopping(patience=3, restore_best_weights=True)])

Epoch 1/50
498/498 [==============================] - ETA: 0s - loss: 1.3031 - accuracy: 0.4030

In [ ]:
# Save the trained model
model.save('/content')